### HW04: Practice with feature engineering, splitting data, and fitting and regularizing linear models

[Ramona Liu kliu268]

### Hello Students:

- Start by downloading HW04.ipynb from this folder. Then develop it into your solution.
- Write code where you see "... your code here ..." below.
  (You are welcome to use more than one cell.)
- If you have questions, please ask them in class, office hours, or piazza. Our TA
  and I are very happy to help with the programming (provided you start early
  enough, and provided we are not helping so much that we undermine your learning).
- When you are done, run these Notebook commands:
  - Shift-L (once, so that line numbers are visible)
  - Kernel > Restart and Run All (run all cells from scratch)
  - Esc S (save)
  - File > Download as > HTML
- Turn in:
  - HW04.ipynb to Canvas's HW04.ipynb assignment
  - HW04.html to Canvas's HW04.html assignment
  - As a check, download your files from Canvas to a new 'junk' folder. Try 'Kernel > Restart
  and Run All' on the '.ipynb' file to make sure it works. Glance through the '.html' file.
- Turn in partial solutions to Canvas before the deadline. e.g. Turn in part 1,
  then parts 1 and 2, then your whole solution. That way we can award partial credit
  even if you miss the deadline. We will grade your last submission before the deadline.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

## 1. Feature engineering (one-hot encoding and data imputation)

### 1a. Read the data from [http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv](http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv).
- Retain only these columns: Survived, Pclass, Sex, Age, SibSp, Parch.
- Display the first 7 rows.

These data are described at [https://www.kaggle.com/competitions/titanic/data](https://www.kaggle.com/competitions/titanic/data) (click on the small down-arrow to see the "Data Dictionary"), which is where they are from.
- Read that "Data Dictionary" paragraph (with your eyes, not python) so you understand what each column represents.

(We used these data before in HW02:
- There we used `df.dropna()` to drop any observations with missing values; here we use data imputation instead.
- There we manually did one-hot encoding of the categorical `Sex` column by making a `Female` column; here we do the same one-hot encoding with the help of pandas's `df.join(pd.get_dummies())`.
- There we used a decision tree; here we use $k$-NN.

We evaluate how these strategies can improve model performance by allowing us to use columns with categorical or missing data.)

In [2]:
df = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/kaggle_titanic_train.csv")
df = df[['Survived','Pclass','Sex','Age','SibSp','Parch']]
df.head(7)

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
5,0,3,male,NaN,0,0
6,0,1,male,54.0,0,0


### 1b. Try to train a $k$NN model to predict $y=$ 'Survived' from $X=$ these features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'.
- Use $k = 3$ and the (default) euclidean metric.
- Notice at the bottom of the error message that it fails with the error "ValueError: could not convert string to float: 'male'".
- Comment out your .fit() line so the cell can run without error.

In [3]:
X = df[['Pclass','Sex','Age','SibSp','Parch']].to_numpy()
y = df['Survived'].to_numpy()
k = 3
knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
#knn.fit(X, y)

### 1c. Try to train again, this time without the 'Sex' feature.
- Notice that it fails because "Input contains NaN".
- Comment out your .fit() line so the cell can run without error.
- Run `X.isna().any()` (where X is the name of your DataFrame of features) to see that
  the 'Age' feature has missing values. (You can see the first missing value in
  the sixth row that you displayed above.)

In [4]:
X1c = df[['Pclass','Age','SibSp','Parch']]#.to_numpy()
y = df['Survived'].to_numpy()
#knn.fit(X1c,y)
X1c.isna().any()

Pclass    False
Age        True
SibSp     False
Parch     False
dtype: bool

### 1d. Train without the 'Sex' and 'Age' features.
- Report accuracy on the training data with a line of the form
  `Accuracy on training data is  0.500` (0.500 may not be correct).

In [5]:
X1d = df[['Pclass','SibSp','Parch']].to_numpy()
y = df['Survived'].to_numpy()
knn.fit(X1d,y)
print(f'Accuracy on training data is {round(knn.score(X1d,y),3)}')

Accuracy on training data is 0.633


### 1e.  Use one-hot encoding
to include a binary 'male'  feature made from the 'Sex' feature. (Or include a binary 'female'
feature, according to your preference. Using both is unnecessary since either is the logical
negation of the other.) That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male'.
- Use pandas's df.join(pd.get_dummies())`.
- Report training accuracy as before.

In [6]:
df1e = df.join(pd.get_dummies(df.Sex, drop_first=False)[['male']])
X1e = df1e[['Pclass','SibSp','male','Parch']].to_numpy()
y = df1e['Survived'].to_numpy()
knn.fit(X1e,y)
print(f'Accuracy on training data is {round(knn.score(X1e,y),3)}')

Accuracy on training data is 0.744


### 1f. Use data imputation
to include an 'age' feature made from 'Age' but replacing each missing value with the median
of the non-missing ages. That is, train on these features: 'Pclass', 'SibSp', 'Parch', 'male',
'age'.

- Report training accuracy as before.

In [7]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean', fill_value=None)
age = imp.fit_transform(df1e.Age.to_numpy().reshape(-1, 1)) # .reshape(-1, 1) says go to 2D, deducing #rows
df1e[['Age']]=age 
#df1e.Age.isna().any() #make sure there's no n/a values in age
X1f = df1e[['Pclass','SibSp','male','Parch','Age']].to_numpy()
y = df1e['Survived'].to_numpy()
knn.fit(X1f,y)
print(f'Accuracy on training data is {round(knn.score(X1f,y),3)}')

Accuracy on training data is 0.874


## 2. Explore model fit, overfit, and regularization in the context of multiple linear regression

### 2a. Prepare the data:
- Read [http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv](http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv) into a DataFrame.
- Set a variable `X` to the subset consisting of all columns except `mpg`.
- Set a variable `y` to the `mpg` column.
- Use `train_test_split()` to split `X` and `y` into `X_train`, `X_test`, `y_train`, and `y_test`.
  - Reserve half the data for training and half for testing.
  - Use `random_state=0` to get reproducible results.

In [8]:
cars = pd.read_csv("http://www.stat.wisc.edu/~jgillett/451/data/mtcars.csv")
X_columns = cars.loc[:, ~cars.columns.isin(['Unnamed: 0', 'mpg'])].columns
X = cars[X_columns]#.to_numpy()
y = cars['mpg']#.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

### 2b. Train three models on the training data and evaluate each on the test data:
- `LinearRegression()`
- `Lasso()`
- `Ridge()`

The evaluation consists in displaying MSE$_\text{train}, $ MSE$_\text{test}$, and the coefficients $\mathbf{w}$ for each model.

In [9]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.5, random_state=0)
models = [linear_model.LinearRegression(), linear_model.Lasso(max_iter=2000), linear_model.Ridge()] 
df = pd.DataFrame(columns=['model', 'MSE_train', 'MSE_test','.coef_'])
for model in models:
    model.fit(X_train, y_train)
    MSE_train = (1/y_train.size) * np.sum((y_train - model.predict(X_train))**2)
    MSE_test = (1/y_test.size)  * np.sum((y_test - model.predict(X_test))**2)
    df = df.append(pd.DataFrame({'model': model, 'MSE_train': MSE_train,
                                     'MSE_test': MSE_test, '.coef_': [np.round(model.coef_, 2)]}), # round for display
                       ignore_index=True)
pd.set_option('display.precision', 2)
print(df)
#print(len(df['.coef_'][0]))=10 proves that D=10 for this problem since there are 10 columns being variables of X
#though it shows a warning for frame.append and suggests using pandas.concat, df.concat does not work now.

                  model MSE_train MSE_test  \
0    LinearRegression()      0.39    30.23   
1  Lasso(max_iter=2000)      5.69    12.99   
2               Ridge()      1.99     11.2   

                                              .coef_  
0  [1.08, 0.03, 0.03, 3.13, -7.34, 3.93, -4.09, -...  
1  [-0.0, -0.04, -0.02, 0.0, -0.0, 0.0, 0.0, 0.0,...  
2  [-0.01, -0.01, 0.0, 0.79, -3.22, 1.1, -0.48, 0...  


/var/folders/6k/mswn4b2d0l700r7q972_l3480000gn/T/ipykernel_2766/695032308.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'model': model, 'MSE_train': MSE_train,
/var/folders/6k/mswn4b2d0l700r7q972_l3480000gn/T/ipykernel_2766/695032308.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'model': model, 'MSE_train': MSE_train,
/var/folders/6k/mswn4b2d0l700r7q972_l3480000gn/T/ipykernel_2766/695032308.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'model': model, 'MSE_train': MSE_train,


### 2c. Answer a few questions about the models:
- Which one best fits the training data?
- Which one best fits the test data?
- Which one does feature selection by setting most coefficients to zero?- 

LinearRegression best fits the training data; Ridge best fits the test data, Lasso sets coefficeints to zero.